In [14]:
import pandas as pd              # For data manipulation and analysis
import numpy as np               # For numerical operations
import matplotlib.pyplot as plt  # For data visualization (basic)
import seaborn as sns            # For advanced data visualization
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [6]:
#Load CSV
df=pd.read_csv('premier_league_combined.csv')
df.head()

,Season,SourceFile,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,1993-94,season-9394.csv,1993-08-14,Arsenal,Coventry,0,3,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-94,season-9394.csv,1993-08-14,Aston Villa,QPR,4,1,H,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-94,season-9394.csv,1993-08-14,Chelsea,Blackburn,1,2,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-94,season-9394.csv,1993-08-14,Liverpool,Sheffield Weds,2,0,H,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-94,season-9394.csv,1993-08-14,Man City,Leeds,1,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# --- KEEP ONLY COMPLETE SEASONS
performance_features = ['HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']

# Identify incomplete seasons (those with missing performance data)
seasons_with_missing = df.loc[df[performance_features].isnull().any(axis=1), 'Season'].unique()
print("Seasons containing missing performance features:", seasons_with_missing)

# Keep only seasons that are NOT in the list (the ~ symbol inverts the condition)
df = df[~df['Season'].isin(seasons_with_missing)].reset_index(drop=True)

print(f" Removed {len(seasons_with_missing)} incomplete seasons.")
print("New dataset shape:", df.shape)


Seasons containing missing performance features: ['1993-94' '1994-95' '1995-96' '1996-97' '1997-98' '1998-99' '1999-00']
 Removed 7 incomplete seasons.
New dataset shape: (9500, 24)


In [8]:
df.shape
df.head()


,Season,SourceFile,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2000-01,season-0001.csv,2000-08-19,Charlton,Man City,4,0,H,2.0,0.0,...,14.0,4.0,13.0,12.0,6.0,6.0,1.0,2.0,0.0,0.0
1,2000-01,season-0001.csv,2000-08-19,Chelsea,West Ham,4,2,H,1.0,0.0,...,10.0,5.0,19.0,14.0,7.0,7.0,1.0,2.0,0.0,0.0
2,2000-01,season-0001.csv,2000-08-19,Coventry,Middlesbrough,1,3,A,1.0,1.0,...,3.0,9.0,15.0,21.0,8.0,4.0,5.0,3.0,1.0,0.0
3,2000-01,season-0001.csv,2000-08-19,Derby,Southampton,2,2,D,1.0,2.0,...,4.0,6.0,11.0,13.0,5.0,8.0,1.0,1.0,0.0,0.0
4,2000-01,season-0001.csv,2000-08-19,Leeds,Everton,2,0,H,2.0,0.0,...,8.0,6.0,21.0,20.0,6.0,4.0,1.0,3.0,0.0,0.0


In [9]:
print("Remaining missing values per column:")
print(df.isnull().sum())


Remaining missing values per column:
Season        0
SourceFile    0
Date          0
HomeTeam      0
AwayTeam      0
FTHG          0
FTAG          0
FTR           0
HTHG          0
HTAG          0
HTR           0
Referee       0
HS            0
AS            0
HST           0
AST           0
HF            0
AF            0
HC            0
AC            0
HY            0
AY            0
HR            0
AR            0
dtype: int64


In [10]:
# --- REMOVE NON-RELEVANT COLUMNS
cols_to_drop = ['HTHG', 'HTAG', 'HTR', 'Referee']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

print(" Dropped columns:", cols_to_drop)
print("New dataset shape:", df.shape)

 Dropped columns: ['HTHG', 'HTAG', 'HTR', 'Referee']
New dataset shape: (9500, 20)


In [11]:
# --- FEATURE ENGINEERING
# Goal Difference (Home - Away) → measures dominance in the match
df['GoalDiff'] = df['FTHG'] - df['FTAG']

# Total Goals (Home + Away) → measures match intensity
df['TotalGoals'] = df['FTHG'] + df['FTAG']

# Shot Accuracy (ratio of shots on target to total shots)
df['ShotAcc_H'] = df.apply(lambda x: x['HST']/x['HS'] if x['HS'] > 0 else 0, axis=1)
df['ShotAcc_A'] = df.apply(lambda x: x['AST']/x['AS'] if x['AS'] > 0 else 0, axis=1)

print("New features created: GoalDiff, TotalGoals, ShotAcc_H, ShotAcc_A")
df[['HomeTeam','AwayTeam','GoalDiff','TotalGoals','ShotAcc_H','ShotAcc_A']].head()

New features created: GoalDiff, TotalGoals, ShotAcc_H, ShotAcc_A


,HomeTeam,AwayTeam,GoalDiff,TotalGoals,ShotAcc_H,ShotAcc_A
0,Charlton,Man City,4,4,0.823529,0.500000
1,Chelsea,West Ham,2,6,0.588235,0.416667
2,Coventry,Middlesbrough,-2,4,0.500000,0.562500
3,Derby,Southampton,0,4,0.666667,0.461538
4,Leeds,Everton,2,2,0.470588,0.500000


In [13]:
# --- LABEL ENCODING FOR SIMPLE CATEGORIES

label_cols = ['Season', 'FTR']
for col in label_cols:
    if col in df.columns:
        encoder = LabelEncoder()
        df[col + '_encoded'] = encoder.fit_transform(df[col])
        print(f"Encoded column: {col} -> {col}_encoded")

# --- ONE-HOT ENCODING FOR TEAM NAMES ---
# Convert HomeTeam and AwayTeam into dummy variables
df = pd.get_dummies(df, columns=['HomeTeam','AwayTeam'], drop_first=True)

print(" Encoding completed. New shape:", df.shape)

Encoded column: Season -> Season_encoded
Encoded column: FTR -> FTR_encoded
 Encoding completed. New shape: (9500, 114)


In [15]:
# --- STANDARDIZATION OF NUMERIC FEATURES ---
numeric_features = ['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC',
                    'HY','AY','HR','AR','GoalDiff','TotalGoals','ShotAcc_H','ShotAcc_A']

scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df[numeric_features]),
    columns=[c + '_z' for c in numeric_features],
    index=df.index
)

# Replace original numeric columns by standardized versions
df = pd.concat([df.drop(columns=numeric_features), df_scaled], axis=1)

print(" Standardization completed. Dataset ready for analysis.")
df.head()

 Standardization completed. Dataset ready for analysis.


,Season,SourceFile,Date,FTR,Season_encoded,FTR_encoded,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,...,HC_z,AC_z,HY_z,AY_z,HR_z,AR_z,GoalDiff_z,TotalGoals_z,ShotAcc_H_z,ShotAcc_A_z
0,2000-01,season-0001.csv,2000-08-19,H,0,2,False,False,False,False,...,-0.011986,0.446826,-0.382680,0.163747,-0.247221,-0.294277,2.007458,0.767337,2.117276,0.306274
1,2000-01,season-0001.csv,2000-08-19,H,0,2,False,False,False,False,...,0.309683,0.810223,-0.382680,0.163747,-0.247221,-0.294277,0.906592,1.964824,0.810776,-0.119557
2,2000-01,season-0001.csv,2000-08-19,A,0,0,False,False,False,False,...,0.631353,-0.279969,2.905096,0.939606,3.700001,-0.294277,-1.295140,0.767337,0.320839,0.625647
3,2000-01,season-0001.csv,2000-08-19,D,0,1,False,False,False,False,...,-0.333656,1.173620,-0.382680,-0.612112,-0.247221,-0.294277,-0.194274,0.767337,1.246276,0.109737
4,2000-01,season-0001.csv,2000-08-19,H,0,2,False,False,False,False,...,-0.011986,-0.279969,-0.382680,0.939606,-0.247221,-0.294277,0.906592,-0.430150,0.157526,0.306274
